In [ ]:
# Import pandas if you dont have instated you can use requirements.txt to install it
import pandas as pd


In [ ]:
# Pleas enter the date from which you want to start the reports 
start_reports='2023-10-10'

# Choose language 'ENG' for English or 'POL' Polish default language is English 
language_choice = 'POL'

if language_choice == 'POL':
    labels = ['Nadczo', 'Po śniadaniu', 'Po obiedzie', 'Po kolacji']
    output_name = f'Zbiorcze zestawienie poziomu cukru od dnia {start_reports}.csv'
else:
    labels = ['Fasting', 'After breakfast', 'After lunch', 'After dinner']
    output_name = f'Summary of sugar levels from date {start_reports}.csv'

# Chosen start reports date language and labels
print(f'''
Start reports date is: {start_reports},
language is: {language_choice}, 
chosen labels is: {', '.join(labels)}
''')

In [ ]:
# Read data form Accu-Check instant glucose meter 
df_import = pd.read_csv('Sample_data.csv')


In [ ]:
# Print out columns name
df_import.columns

In [ ]:
# Choose to necessary columns to reduce size Data Frame
df_selected_columns = df_import[['Date','Time', 'Blood Sugar Measurement (mg/dL)']]

# Rename columns names to keep naming convention in Python
df_convert_format = df_selected_columns.rename(columns={'Date': 'date', 'Time': 'time', 'Blood Sugar Measurement (mg/dL)': 'sugar'})

# Convert 'date', 'time' column to date format and create datetime combining 'date' and 'time'
df_convert_format['date'] = pd.to_datetime(df_convert_format['date'])
df_convert_format['time'] = pd.to_datetime(df_convert_format['time'], format='%I:%M:%S %p').dt.time
df_convert_format['datetime'] = df_convert_format['date'] + pd.to_timedelta(df_convert_format['time'].apply(lambda x: x.strftime('%H:%M:%S')))

# Print out result
df_convert_format

In [ ]:
# Filter out unnecessary data (older than "start_reports" and containing non-null values in the "sugar" column)
filtered_df = df_convert_format[(df_convert_format['date'] >= start_reports) & (~df_convert_format['sugar'].isna())]
filtered_df

In [ ]:
# Convert 'sugar' column to integer type hint it is necessary to 'sugar' column not contains null value in previous step
filtered_df.loc[:, 'sugar'] = filtered_df['sugar'].astype(int)

# Display the converted data
df = filtered_df.sort_values(by=['date', 'time', 'sugar'], ascending=[False, False, True])

# Print first 20 values
df.head(20)

In [ ]:
# Get fasting data 
df_fasting = df.drop_duplicates('date', keep='last')
print(f'Data size before removing to late time {df_fasting.shape[0]} ')

# Remove if the first blood sugar measurement was after 11 am. 
df_fasting = df_fasting[df_fasting['time'] <= pd.to_datetime('11:00:00').time()]

# Label as 'Fasting'
df_fasting['label'] = labels[0]

# Size after data 
print(f'Data size after removing to late time {df_fasting.shape[0]}')

# Print data 
df_fasting

In [ ]:
# Get After dinner measurement we assume that is after 7 pm. 
df_dinner = df.drop_duplicates('date', keep='first').loc[df['time']>= pd.to_datetime('19:00:00').time()]

# Label data as 'After dinner'
df_dinner['label'] = labels[3]

# Print data 
df_dinner

In [ ]:
# Merge data with fasting and after dinner to reduce Data Frame data categorized to one of this group on next step  
merged_df_dinner = df.merge(df_dinner[['date', 'datetime']], on='date', suffixes=('_df', '_dinner'))
merged_df_dinner_fasting = merged_df_dinner.merge(df_fasting[['date', 'datetime']].rename(columns={'datetime': 'datetime_fasting'}), on='date', how='left')

# Print first 20 rows
merged_df_dinner_fasting.head(20)


In [ ]:
# Get lower sugar level 2 hours after lunch we assuming it will be between 6-1 hours before "After dinner" measurement 
df_lunch= merged_df_dinner_fasting[(merged_df_dinner_fasting['datetime_dinner'] - merged_df_dinner_fasting['datetime_df'] < pd.Timedelta(hours=6)) & 
                                    (merged_df_dinner_fasting['datetime_dinner'] - merged_df_dinner_fasting['datetime_df'] >= pd.Timedelta(hours=1))]

# Sort values date and sugar ascending
df_lunch = df_lunch.sort_values(by=['date', 'sugar'], ascending=[True, True])

# Get lower sugar level from each day
df_lunch_get_better = df_lunch.groupby('date').first().reset_index()

# Label as after lunch
df_lunch_get_better['label'] = labels[2]

# Print out better result after lunch
df_lunch_get_better

In [ ]:
# Get lower sugar level 2 hours after breakfast we assuming it will be between 6-1 hours after "Fasting" measurement 
df_breakfast= merged_df_dinner_fasting[(merged_df_dinner_fasting['datetime_df'] - merged_df_dinner_fasting['datetime_fasting'] < pd.Timedelta(hours=6)) & 
                                       (merged_df_dinner_fasting['datetime_df'] - merged_df_dinner_fasting['datetime_fasting'] > pd.Timedelta(hours=1))]

# Sort values date and sugar ascending
df_breakfast = df_breakfast.sort_values(by=['date', 'sugar'], ascending=[True, True])

# Get lower sugar level from each day after breakfast
df_breakfast_get_better = df_breakfast.groupby('date').first().reset_index()

# Label as after breakfast
df_breakfast_get_better['label'] = labels[1]

# Print out better result after breakfast
df_breakfast_get_better

In [ ]:
# Merge all selected Data Frame in to one
df = pd.concat([df_dinner, df_breakfast_get_better, df_lunch_get_better, df_fasting])

# Filling missing data and for 2 labels categories (after merging was added suffix) if datetime doesn't contain data in 'datetime'
df['datetime'] = df['datetime'].fillna(df['datetime_df'])

# Keep only selected columns 
df_selected_columns = df[['date', 'time', 'sugar', 'datetime', 'label']].copy()

# Print before export data
df_selected_columns

In [ ]:
df_selected_columns['sugar'] = df_selected_columns['sugar'].astype(int)
df_selected_columns

In [ ]:
# Create pivot table using sugar level data per day, to remove duplicates used aggregation function take minimal
pivot_table = df_selected_columns.pivot_table(index='date', columns='label', values='sugar', aggfunc='min')

# Rearrange order columns to have as in labels list
pivot_table = pivot_table.reindex(columns=labels)

# Save as output file
pivot_table.to_csv(output_name)

# Print out final data
pivot_table

In [ ]:
# Styled table
# set data as %Y-%m-%d format
pivot_table.index = pd.to_datetime(pivot_table.index).strftime('%Y-%m-%d').str[:10]

# Round sugar to integer or put empty string
styled_table = pivot_table.style\
    .format({col: lambda x: f"{int(x):.0f}" if not pd.isna(x) else '' for col in pivot_table.columns})\
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])\
    .set_caption(output_name)\
    .set_properties(**{'color': 'black',
                       'border-color': 'white'})

# Adding background gradient and show table
styled_table_with_gradient = styled_table.background_gradient(cmap='pink_r', axis=None)
styled_table_with_gradient

In [ ]:
# Export to excel:
styled_table_with_gradient.to_excel('Excel_export.xlsx', engine='openpyxl', index=False)
# Export to html:
styled_table_with_gradient.to_html('Html_export.html')
